<center>
    
 # **ACIT 4610 Mandatory 2**
 
 ## **Multi-Objective VRP: NSGA-II vs SPEA2**
 
 ## **Group nr: 4**

</center>


## **1. Data Ingestion and Setup**

- Import libraries and dependencies
- Load instance(s)
- Build distance matrix and basic helpers


In [ ]:

import time, ast, math, random
from typing import NamedTuple, List, Tuple, Dict
import numpy as np
import pandas as pd
import vrplib


In [ ]:
#Load instance
instance_small = vrplib.read_instance("A-n32-k5.vrp")
instance_medium = vrplib.read_instance("B-n78-k10.vrp")
instance_large = vrplib.read_instance("X-n101-k25.vrp")

#get capacity, demands, coordinates and depot
def get_instance_details(instance: Dict) -> Tuple[int, List[int], Dict[int, Tuple[float, float]], List[int]]:
    capacity = instance["capacity"]            # vehicle capacity Q
    demands = instance["demand"]               # list/array of demands
    coords = instance["node_coord"]            # dict {id: (x,y)}
    depot = instance["depot"]                  # depot node id(s)
    return capacity, demands, coords, depot

capacity_small, demands_small, coords_small, depot_small = get_instance_details(instance_small)
capacity_medium, demands_medium, coords_medium, depot_medium = get_instance_details(instance_medium)    
capacity_large, demands_large, coords_large, depot_large = get_instance_details(instance_large)

#print("Small instance:") 
#print(f"  Capacity: {capacity_small}")

def distance_matrix(coords: List) -> np.ndarray:
    n = len(coords)  # includes depot
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            dist = math.dist(coords[i+1], coords[j+1])  # coords keys start from 1
            dist_matrix[i][j] = dist
            dist_matrix[j][i] = dist  # symmetry
    return dist_matrix




## **2. Chromosome Representation and VRP Instances**

- Define instances (small, medium, large)
- Individual representation (routes/encoding)
- Decoding helpers


## **3. Multi-Objective Problem Definition**

- Objectives: minimize total distance, improve route balance (e.g., std dev of route loads/lengths)
- Constraints: capacity, feasibility handling
- Evaluation function


## **4. Evolutionary Algorithms**

### **4.1 NSGA-II Setup**
- Initialization, crossover/mutation operators, parameters

### **4.2 SPEA2 Setup**
- Strength assignment, density estimation, parameters


## **5. Experiments and Metrics**

- Multiple independent runs (seeds)
- Compute Hypervolume (HV), Inverted Generational Distance (IGD), spacing/diversity
- Record runtimes


## **6. Visualization and Discussion**

- Plot Pareto fronts
- Compare NSGA-II vs SPEA2
- Key insights and limitations
